### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline


# Read in the MovieTweetings dataset originally taken from https://github.com/sidooms/MovieTweetings/tree/master/latest
movies = pd.read_csv('movies.dat', delimiter='::', header=None, names=['movie_id', 'movie', 'genre'], dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('ratings.dat', delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
movies.head()

,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
reviews.head()

,user_id,movie_id,rating,timestamp
0,1,0114508,8,1381006850
1,2,0208092,5,1586466072
2,2,0358273,9,1579057827
3,2,10039344,5,1578603053
4,2,6751668,9,1578955697


In [4]:
movies['movie'].nunique(), movies['movie_id'].nunique(), reviews['movie_id'].nunique()

(35416, 35479, 35479)

In [5]:
movies['genre'].dropna(axis=0).nunique()

2736

In [6]:
reviews['rating'].value_counts()

8     208405
7     193316
9     122477
6     112642
10    101961
5      64856
4      26589
3      14543
1      10166
2       8650
0        261
Name: rating, dtype: int64

In [7]:
reviews['rating'].count(), reviews['rating'].shape[0], reviews[reviews['rating'].isnull()==True].shape[0]

(863866, 863866, 0)

In [8]:
reviews['rating'].mean()

7.3158776939942074

In [9]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': reviews['movie_id'].dropna(axis=0).nunique(),
'The number of ratings in the dataset': reviews['rating'].dropna(axis=0).size,
'The number of different genres': 28, #movies['genre'].dropna(axis=0).nunique(),
'The number of unique users in the dataset': reviews['user_id'].dropna(axis=0).nunique(),
'The number missing ratings in the reviews dataset': reviews[reviews['rating'].isnull()==True].shape[0],
'The average rating given across all ratings': round(reviews['rating'].mean(),0),
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

print(reviews['movie_id'].dropna(axis=0).nunique())
print(reviews['rating'].dropna(axis=0).size)
print(movies['genre'].dropna(axis=0).nunique())
print(reviews['user_id'].dropna(axis=0).nunique())

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea
t.q1_check(dict_sol1)

35479
863866
2736
67353
That looks good to me!


In [10]:
genres = []
for val in movies.genre:
    try:
        genres.extend(val.split('|'))
    except AttributeError:
        pass
    
print(genres)
genres = set(genres)
print(genres)

['Documentary', 'Short', 'Documentary', 'Short', 'Documentary', 'Short', 'Short', 'Horror', 'Short', 'Comedy', 'Horror', 'Short', 'Action', 'Adventure', 'Comedy', 'Fantasy', 'Sci-Fi', 'Short', 'Action', 'Crime', 'Western', 'Action', 'Short', 'Short', 'Drama', 'Short', 'Horror', 'Sci-Fi', 'Short', 'Drama', 'Romance', 'Drama', 'History', 'Adventure', 'Drama', 'Fantasy', 'Horror', 'Short', 'Comedy', 'Romance', 'Crime', 'Drama', 'Adventure', 'Drama', 'History', 'Comedy', 'Short', 'Family', 'Fantasy', 'Adventure', 'Comedy', 'Short', 'Comedy', 'Short', 'Comedy', 'Short', 'Comedy', 'Short', 'Comedy', 'Short', 'Comedy', 'Drama', 'Adventure', 'Family', 'Fantasy', 'Comedy', 'Short', 'Comedy', 'Short', 'Comedy', 'Short', 'Drama', 'History', 'War', 'Short', 'Comedy', 'Sport', 'Drama', 'Short', 'Comedy', 'Biography', 'Crime', 'Drama', 'Romance', 'Short', 'Comedy', 'Action', 'Adventure', 'Crime', 'Drama', 'Mystery', 'Thriller', 'Short', 'Comedy', 'Romance', 'Drama', 'Short', 'Comedy', 'Short', 'Come

#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [11]:
cent = []
for val in movies.movie:
    try:
        cent.append(int(val[-5:-1]))
    except AttributeError:
        cent.append(NaN)
        
print(cent)

[1894, 1895, 1896, 1895, 1896, 1896, 1902, 1903, 1903, 1908, 1909, 1910, 1911, 1912, 1911, 1911, 1913, 1914, 1914, 1914, 1914, 1914, 1914, 1914, 1916, 1914, 1914, 1914, 1915, 1915, 1915, 1915, 1915, 1915, 1915, 1915, 1915, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1917, 1917, 1917, 1918, 1918, 1918, 1918, 1919, 1919, 1918, 1919, 1919, 1920, 1920, 1919, 1919, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1921, 1921, 1921, 1921, 1921, 1921, 1921, 1921, 1921, 1922, 1922, 1922, 1922, 1922, 1922, 1922, 1922, 1922, 1923, 1922, 1923, 1923, 1923, 1924, 1923, 1923, 1923, 1923, 1923, 1923, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1924, 1925, 1924, 1924, 1926, 1925, 1925, 1925, 1925, 1925, 1925, 1924, 1925, 1925, 1925, 1925, 1925, 1925, 1925, 1926, 1926, 1926, 1926, 1926, 1926, 1927, 1927, 1926, 1926, 1926, 1926, 1926, 1927, 1927, 1927, 1927, 1927, 1927, 1927, 1928, 1926, 1927, 1927, 1927, 1927, 1927, 1928, 1927, 1927, 1927, 1927, 1927, 1927, 192

In [12]:
create_date = lambda val: val[-5:-1] if val[-1]==')' else np.nan
movies['date'] = movies['movie'].apply(create_date) 
movies.head()

,movie_id,movie,genre,date
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895
4,0000091,Le manoir du diable (1896),Short|Horror,1896


In [13]:
def dump_yr(val):
    if val[:2] == yr:
        return 1
    else:
        return 0

for yr in ["18","19","20"]:
    movies[yr +"00's"] = movies['date'].apply(dump_yr)
    
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0


In [14]:
def dump_genres(val):
    try:
        if val.find(gene) >-1:
            return 1
        else:
            return 0
    except AttributeError:
        return 0

for gene in genres:
    movies[gene] = movies['genre'].apply(dump_genres)
    
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Adventure,Drama,...,Romance,Family,Biography,Musical,Sci-Fi,Thriller,Adult,Mystery,News,Game-Show
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
import datetime

dump_timestamp = lambda val: datetime.datetime.fromtimestamp(int(val)).strftime('%Y-%m-%d %H:%M:%S')
reviews['date'] = reviews['timestamp'].apply(dump_timestamp)
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-05 21:00:50
1,2,0208092,5,1586466072,2020-04-09 21:01:12
2,2,0358273,9,1579057827,2020-01-15 03:10:27
3,2,10039344,5,1578603053,2020-01-09 20:50:53
4,2,6751668,9,1578955697,2020-01-13 22:48:17


In [16]:
def dump_month(val):
    if int(val.split('-')[1]) == mon:
        return 1
    else:
        return 0
    
for mon in range(1, 13, 1):
    reviews['month_'+ str(mon)] = reviews['date'].apply(dump_month)
    
reviews.head(10)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1,0114508,8,1381006850,2013-10-05 21:00:50,0,0,0,0,0,0,0,0,0,1,0,0
1,2,0208092,5,1586466072,2020-04-09 21:01:12,0,0,0,1,0,0,0,0,0,0,0,0
2,2,0358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,0,0,0,0,0,0,0
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,0,0,0,0,0,0,0
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,0,0,0,0,0,0,0
5,2,7131622,8,1579559244,2020-01-20 22:27:24,1,0,0,0,0,0,0,0,0,0,0,0
6,2,7975244,6,1583706305,2020-03-08 22:25:05,0,0,1,0,0,0,0,0,0,0,0,0
7,2,7984734,7,1579949842,2020-01-25 10:57:22,1,0,0,0,0,0,0,0,0,0,0,0
8,2,8367814,8,1586436354,2020-04-09 12:45:54,0,0,0,1,0,0,0,0,0,0,0,0
9,2,8579674,10,1579261830,2020-01-17 11:50:30,1,0,0,0,0,0,0,0,0,0,0,0


In [27]:
get_num_yr = lambda val: val.split('-')[0]
reviews['date'].apply(get_num_yr).value_counts()

2013    170807
2014    156354
2015    123644
2016    112455
2017    105987
2018     81870
2019     74183
2020     38566
Name: date, dtype: int64

In [20]:
def dump_yr(val):
    if int(val.split('-')[0]) == yr:
        return 1
    else:
        return 0
    
for yr in range(2013, 2021, 1):
    reviews['year_'+ str(yr)] = reviews['date'].apply(dump_yr)

reviews.head(10)

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,0114508,8,1381006850,2013-10-05 21:00:50,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,0208092,5,1586466072,2020-04-09 21:01:12,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,2,7131622,8,1579559244,2020-01-20 22:27:24,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,2,7975244,6,1583706305,2020-03-08 22:25:05,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7,2,7984734,7,1579949842,2020-01-25 10:57:22,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,2,8367814,8,1586436354,2020-04-09 12:45:54,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
9,2,8579674,10,1579261830,2020-01-17 11:50:30,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
reviews.to_csv('./reviews_clean.csv')
movies.to_csv('./movies_clean.csv')

In [19]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year